# Sentiment Analysis Of Customer Feedback On Restaurants Using IBM Cloud

In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_csv("zomato.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'zomato.csv'

In [ ]:
data_review = dataset['reviews_list']

In [ ]:
dataset.head()

In [ ]:
dataset.isnull().any()

In [ ]:
data_review

In [ ]:
x = []
y = []

In [ ]:
x

In [ ]:
y

In [ ]:
for row_num in range(0,51717):
    # split the revie text at '()
    lst = data_review[row_num].split("('")
    for i in lst:
        if len(i) > 5:
            if i.find("',") != -1:
                single_rev = i.split("',")
                if len(single_rev[0]) > 2:
                    x.append(single_rev[0])
                if len(single_rev[1]) > 2:    
                    y.append(single_rev[1])

In [ ]:
import nltk
nltk.download("stopwords") 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
# to store the final rating
rating_final = []
# to store cleaned revies
review_final = []

In [ ]:
import re

In [ ]:
for i in range(0,40000):
    data_x = x[i]
    data_x = re.sub('[a-zA-Z]', ' ' , data_x)
    data_x = data_x.split()
    data_x = ''.join(data_x)
    data_x = float(data_x)
    if data_x < 2.5:
        rating_final.append("poor") #poor
    elif data_x >= 2.5 and data_x <= 3.5 :    
        rating_final.append("average") # average
    elif data_x > 3.5:
        rating_final.append("good") #good
        

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
rating_final = le.fit_transform(rating_final)

In [ ]:
rating_final = np.array(rating_final)
rating_final = np.expand_dims(rating_final, axis=1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
one = OneHotEncoder()
rates = one.fit_transform(rating_final).toarray()

In [ ]:
for loop in range(0,40000) : 
    data_y = y[loop]
    data_y = re.sub('[^a-zA-Z]', " ", data_y)
    data_y = data_y.lower()
    data_y = data_y.split()
    data_y = [ps.stem(word) for word in data_y if not word in set(stopwords.words('english'))]
    data_y = ' '.join(data_y)
    review_final.append(data_y)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 20000)
x_final = cv.fit_transform(review_final).toarray()

In [ ]:
# saving the vectorizer which would be used as dictionary.
import pickle
pickle.dump(cv, open('cv.pkl','wb'))

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final,rates, test_size = 0.2, random_state = 0)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(13264,kernel_initializer = "random_uniform",activation ="relu"))

In [ ]:
model.add(Dense(2000,kernel_initializer = "random_uniform",activation ="relu"))

In [ ]:
model.add(Dense(2000,kernel_initializer = "random_uniform",activation ="relu"))

In [ ]:
model.add(Dense(2000,kernel_initializer = "random_uniform",activation ="relu"))

In [ ]:
model.add(Dense(3,kernel_initializer = "random_uniform",activation = "softmax"))

In [ ]:
model.compile("adam","categorical_crossentropy",metrics = ["accuracy"])

In [ ]:
y_pred = model.predict(x_test)

text =  "The food is okay. average place "
text = re.sub('[^a-zA-Z]', ' ',text)
text = text.lower()
text = text.split()
text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
text = ' '.join(text)

y_p = model.predict(cv.transform([text]))

In [ ]:
# saving the model

model.save("zomato_2_analysis.h5")